### Explore the HTML

In [1]:
import requests
web_url = 'http://results.eci.gov.in/DELHITRENDS2020/statewiseU051.htm'
r = requests.get(web_url)

In [2]:
r.text[0:500]

"<!DOCTYPE html>\r\n    <html><head>\r\n    <title>Election Commission of India</title>\r\n    <meta http-equiv='Content-Language' content='en-us'>\r\n    <meta http-equiv='Content-Type' content='text/html; charset=UTF-8'>\r\n    <meta http-equiv='X-UA-Compatible' content='IE=7'>\r\n    <meta http-equiv='Cache-Control' content='no-cache, no-store, must-revalidate'>\r\n    <meta http-equiv='Pragma' content='no-cache'>\r\n    <meta http-equiv='Expires' content='0'>\r\n    <meta http-equiv='refresh' content=abc>\r\n   "

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text, 'html.parser')

In [4]:
soup

<!DOCTYPE html>

<html><head>
<title>Election Commission of India</title>
<meta content="en-us" http-equiv="Content-Language"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=7" http-equiv="X-UA-Compatible"/>
<meta content="no-cache, no-store, must-revalidate" http-equiv="Cache-Control"/>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="0" http-equiv="Expires"/>
<meta content="abc" http-equiv="refresh"/>
<style type="text/css">
          body{
        font-family: Verdana;Arial, Helvetica, sans-serif; font-size: 13px;
        width: 1024px;
        padding: 0px;
        margin: 0 auto;
        background:#105980;
    }
    .wrap-sec{
        text-align:center;
                width:1024px;
    }
    .tabc{
        background-color: #fff;
        width:1024px;
    }
    
    .tabc h1{
         font-size: 26px;
         font-weight: bold;
    }
    .tabc h4{
        font-size: 16px;
        font-weight: bold;
        margin:0;
    }
  

In [6]:
results_rows = soup.find_all('tbody', attrs={'id':'ElectionResult'})[0].find_all('tr', attrs={'style':'font-size:12px;'})
results_rows

[<tr style="font-size:12px;"><td align="left">ADARSH NAGAR</td><td align="center">4</td><td align="left">PAWAN SHARMA</td><td align="left"><table><tbody><tr><td>Aam Aadmi Party</td><td style="vertical-align:top"><span class="tooltip-icon" style="display:block">i</span><div class="tooltip"><h3>Current Assembly Election Result</h3><table><tbody><tr><td>Leading In</td><td>:</td><td>0</td></tr><tr><td>Won In</td><td>:</td><td>62</td></tr><tr><td>Trailing In</td><td>:</td><td>0</td></tr></tbody></table></div></td></tr></tbody></table></td><td align="left">RAJ KUMAR BHATIA</td><td align="left"><table><tbody><tr><td>Bharatiya Janata Party</td>
 <td style="vertical-align:top"><span class="tooltip-icon" style="display:block">i</span><div class="tooltip"><h3>Current Assembly Election Result</h3><table><tbody><tr><td>Leading In</td><td>:0</td><td></td></tr><tr><td>Won In</td><td>:8</td><td></td></tr><tr><td>Trailing In</td><td>:</td><td>0</td></tr></tbody></table>
 </div>
 </td></tr></tbody></tab

In [8]:
len(results_rows)

10

In [21]:
# Const Name
row.contents[11].text

'18287'

### Build the dataset

In [10]:
# web_url = 'http://results.eci.gov.in/DELHITRENDS2020/statewiseU051.htm'
records = []
for no in range(51,58):
#     print(no)
    src = 'http://results.eci.gov.in/DELHITRENDS2020/statewiseU0'+str(no)+'.htm'
    reqURL = requests.get(src)
    bsoup = BeautifulSoup(reqURL.text, 'html.parser')
    data_rows = soup.find_all('tbody', attrs={'id':'ElectionResult'})[0].find_all('tr', attrs={'style':'font-size:12px;'})
    for row in data_rows:
        const_name = row.contents[0].text
        const_no = row.contents[1].text
        lead_cand = row.contents[2].text
        lead_party = row.contents[3].find_all('td')[0].text
        trail_cand = row.contents[4].text
        trail_party = row.contents[5].find_all('td')[0].text
        margin = row.contents[7].text
        status = row.contents[8].text
        win_cand = row.contents[9].text
        win_party = row.contents[10].text
        win_margin = row.contents[11].text
        records.append((const_name, const_no, lead_cand, lead_party, trail_cand, trail_party, margin, status, win_cand, win_party))

In [11]:
records[0:2]

[('ADARSH NAGAR',
  '4',
  'PAWAN SHARMA',
  'Aam Aadmi Party',
  'RAJ KUMAR BHATIA',
  'Bharatiya Janata Party',
  '1589',
  'Result Declared'),
 ('AMBEDKAR NAGAR(SC)',
  '48',
  'AJAY DUTT',
  'Aam Aadmi Party',
  'KHUSHIRAM CHUNAR',
  'Bharatiya Janata Party',
  '28327',
  'Result Declared')]

In [12]:
len(records)

70

In [14]:
import pandas as pd

df = pd.DataFrame(records, columns=['const_name', 'const_no', 'lead_cand', 'lead_party', 'trail_cand', 'trail_party', 'margin', 'status'])

In [15]:
df.tail()

,const_name,const_no,lead_cand,lead_party,trail_cand,trail_party,margin,status
65,BALLIMARAN,22,IMRAN HUSSAIN,Aam Aadmi Party,LATA,Bharatiya Janata Party,36172,Result Declared
66,BAWANA(SC),7,JAI BHAGWAN,Aam Aadmi Party,RAVINDER KUMAR,Bharatiya Janata Party,11526,Result Declared
67,BIJWASAN,36,BHUPINDER SINGH JOON,Aam Aadmi Party,SAT PRAKASH RANA,Bharatiya Janata Party,753,Result Declared
68,BURARI,2,SANJEEV JHA,Aam Aadmi Party,SHAILENDRA KUMAR,Janata Dal (United),88158,Result Declared
69,CHANDNI CHOWK,20,PARLAD SINGH SAWHNEY,Aam Aadmi Party,SUMAN KUMAR GUPTA,Bharatiya Janata Party,29584,Result Declared


### Convert to CSV

In [16]:
df.to_csv('output/u05_constituencywise_2020.csv', index=False, encoding='utf-8')